In [12]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

# Import py module to access CRUD class 
from CS_340_Mod4_Milestone import AnimalShelter

# Allows image to be imported 
import base64

###########################
# Data Manipulation / Model
###########################
# Login information for the AAC account 
username = "aacuser"
password = "aacpass"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))

                               
#########################
# Dashboard Layout / View
#########################
app = JupyterDash('KathrynMcNeil')

# Photo of Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    # My unique identifier 
    html.Center(html.B(html.H1('Kathryn McNeil\'s SNHU CS-340 Dashboard'))),
    html.Hr(),
    # HTML image tag to dispaly photo in dashboard 
    html.Img(id='customer-image', src='data:image/png;base64,{}'.format(encoded_image.decode()), alt='customer image'),
    html.Div(
        # Filters to search for dogs of a specific type 
        dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'reset'},
        ],
        value='reset'
        )
    ),
    html.Hr(),
    # This is the data table where all the animal information will be displayed 
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # Features for the interactive data table making it user-friendly for the client
        editable=False, 
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=15,
    ),
    html.Br(),
    html.Hr(),
    # Formatting the charts so it is easier to view both at the same time 
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
   
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
# Functionality for filtering
# Each filter specifies gender, neutering status, breeds, as well as age 
def update_dashboard(filter_type):
        # Filter for a water rescue dog
        if filter_type == 'water':
            df = pd.DataFrame.from_records(shelter.read({
                'animal_type': 'Dog',
                'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
                'sex_upon_outcome': 'Intact Female',
                'age_upon_outcome_in_weeks': {'$gte': 26.0, '$lte': 156.0}
            }))
        
        # Filter for a mountain rescue dog
        elif filter_type == 'mountain':
            df = pd.DataFrame.from_records(shelter.read({
                'animal_type': 'Dog',
                'breed': {'$in': ['German Shepard','Alaskan Malamute','Old English Sheepdog', 
                                  'Siberian Husky', 'Rottweiler']},
                'sex_upon_outcome': 'Intact Male',
                'age_upon_outcome_in_weeks': {'$gte':26.0, '$lte':156.0}
            }))
                               
        # Filter for a disaster or individual tracking dog 
        elif filter_type == 'disaster':
            df = pd.DataFrame.from_records(shelter.read({
                'animal_type': 'Dog',
                'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 
                                  'Bloodhound', 'Rottweiler']},
                'sex_upon_outcome': 'Intact Male',
                'age_upon_outcome_in_weeks': {'$gte':20.0, '$lte':300.0}
            }))
                               
        # No filter, this removes the filter and displays all the animals
        else: 
            df = pd.DataFrame.from_records(shelter.read({}))                      

        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        return (data,columns)

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
# Code for a visual graph 
def update_graphs(viewData):
    # Setting dff to the data frames data 
    dff = pd.DataFrame.from_dict(viewData)
    
    # Returns a graph with data from the data frame 
    return [
        dcc.Graph(            
            figure = px.histogram(dff, x='breed', title='Breeds')
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
# Geolocation chart
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal's Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

app

done
